In [1]:
import json
import os
from argparse import ArgumentParser, Namespace
from functools import reduce
from glob import glob

#import cv2
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchvision.utils import draw_segmentation_masks

from sidewalk_widths_extractor import Trainer, seed_all
from sidewalk_widths_extractor.dataset import SatelliteDataset
from sidewalk_widths_extractor.modules.seg import SegModule
from sidewalk_widths_extractor.utilities import get_device
from sidewalk_widths_extractor.utilities.io import mkdir

: 

: 

In [ ]:
device = get_device()
print("using", device)

In [ ]:
model_path = "/content/drive/MyDrive/IDP/sidewalk-widths-extractor/logs/hp_d320_resume/19-09-2022 13-46-10 hp_d80_resume 23/checkpoints/best_network.pth.tar"
settings_path = "/content/drive/MyDrive/IDP/sidewalk-widths-extractor/logs/hp_d320_resume/19-09-2022 13-46-10 hp_d80_resume 23/settings.json"

In [ ]:
settings = None
with open(settings_path) as file:
  settings = json.load(file)

module = SegModule(
        settings["module"]["network"]["id"],
        settings["module"]["network"]["params"],
        settings["module"]["optimizer"]["id"],
        settings["module"]["optimizer"]["params"],
        settings["module"]["criterion"]["id"],
        settings["module"]["criterion"]["params"],
        device=device,
        save_network_checkpoint=False,
        save_optimizer_checkpoint=False,
    )


module.load({"network": model_path})

In [ ]:
tensor_transform = T.ToTensor()
image_transform = T.ToPILImage()

In [ ]:
def segment(img):
  image = tensor_transform(img)
  pred = module.infer(image).detach().cpu()
  
  image = (image * 255).type(torch.uint8)
  pred = pred.type(torch.bool)

  seg = draw_segmentation_masks(image, pred, alpha=0.3, colors="blue")

  seg = image_transform(seg)

  return seg

In [ ]:
demo = gr.Interface(segment, gr.Image(shape=(256, 256)), "image")
demo.launch()